In [15]:
"""
Created by Terry (terrydurunkai@gmail.com) on 2022/10/7.
"""

from pyspark import SparkContext


sc = SparkContext.getOrCreate()
words = sc.textFile("gutenberg-1G.txt").flatMap(lambda line: line.split(" "))
# words = sc.textFile("5000-8.txt").flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)

In [16]:
# words = {}
# with open("output/part-00000", mode="r") as f:
#     lines = f.readlines()
#     for line in lines:
wordsMap = wordCounts.collectAsMap()
resultMap = {}
minCount = 1000
maxCount = 0
minWord = []
maxWord = []
for key in wordsMap.keys():
    if key == "" or key == "\'\'" or wordsMap[key] == 15211222:
        continue
    flag = True
    for c in key:
        if ord(c) < 33 or ord(c) > 122:
            flag = False
            break
    if flag:
        resultMap[key] = wordsMap[key]
        if wordsMap[key] <= minCount:
            if wordsMap[key] == minCount:
                minWord.append(key)
            else:
                minWord = [key]
                minCount = wordsMap[key]
        if wordsMap[key] >= maxCount:
            if wordsMap[key] == maxCount:
                maxWord.append(key)
            else:
                maxWord = [key]
                maxCount = wordsMap[key]
# print(minWord, end="")
# print(" is the word(s) with the minimum count of %d" % (minCount))
# print(maxWord, end="")
# print(" is the word(s) with the maximum count of %d" % (maxCount))

In [18]:
with open("1G_min_max.txt", mode="w") as f:
    outputStr = str(minWord) + " is the word(s) with the minimum count of " + str(minCount) + "\n" + str(maxWord) + " is the word(s) with the maximum count of " + str(maxCount)
    f.write(outputStr)
wordCounts = sc.parallelize([resultMap])
wordCounts.saveAsTextFile("output2/")